In [1]:
import pandas as pd

In [2]:
ava_df = pd.read_csv('AVA.txt', sep=' ', names=['Image ID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Semantic ID 1', 'Semantic ID 2', 'Challenge ID'])
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
1,953619,0,1,5,17,38,36,15,6,5,1,1,22,1396
2,953958,10,7,15,26,26,21,10,8,1,2,1,21,1396
3,954184,0,0,4,8,41,56,10,3,4,0,0,0,1396
4,954113,0,1,4,6,48,37,23,5,2,2,15,21,1396
5,953980,0,3,6,15,57,39,6,1,1,1,22,38,1396


In [3]:
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Filter Bad Images

In [ ]:
import numpy as np

# Catch all warnings too
import warnings
warnings.filterwarnings("error")

rows_to_delete = []
for i, row in tqdm(ava_df.iterrows()):
    try:
        im = Image.open('images/{}.jpg'.format(row['Image ID']))
        im.verify()
        im.close()
    except:
        print("Row {} is bad".format(i))
        rows_to_delete.append(i)   

609it [00:00, 2682.43it/s]

Row 1 is bad
Row 2 is bad
Row 3 is bad
Row 4 is bad
Row 5 is bad
Row 6 is bad
Row 7 is bad
Row 8 is bad
Row 9 is bad
Row 10 is bad


1545it [00:00, 3435.21it/s]Exception ignored in: <_io.FileIO name='images/691230.jpg' mode='rb' closefd=True>
5329it [00:01, 4800.30it/s]

Row 4754 is bad


12162it [00:02, 4265.58it/s]Exception ignored in: <_io.FileIO name='images/759039.jpg' mode='rb' closefd=True>
13154it [00:02, 4597.49it/s]

...

115567it [12:30, 66.18it/s]Exception ignored in: <_io.FileIO name='images/772220.jpg' mode='rb' closefd=True>
115582it [12:30, 76.99it/s]

Row 115574 is bad


127030it [15:20, 85.85it/s]Exception ignored in: <_io.FileIO name='images/501015.jpg' mode='rb' closefd=True>
127039it [15:20, 71.07it/s]

Row 127032 is bad


128807it [15:46, 72.39it/s]

In [ ]:
proc_ava_df = ava_df.drop(rows_to_delete)
print('Removed {} rows'.format(len(rows_to_delete)))

In [ ]:
proc_ava_df.to_csv('proc_AVA.csv', index=False)

# Calculate Mean Score

In [ ]:
ava_df = pd.read_csv('proc_AVA.csv')

In [ ]:
ava_df.head()

In [ ]:
import numpy as np

def compute_mean_score(row):
    prob_distr = row[[1,2,3,4,5,6,7,8,9,10]].values
    prob_distr = prob_distr / prob_distr.sum()
    weights = np.arange(10) + 1
    mean_score = (prob_distr * weights).sum()
    return mean_score

In [ ]:
%%time
ava_df['Mean Score'] = ava_df.apply(compute_mean_score, axis=1)

In [ ]:
ava_df.head()

In [ ]:
ava_df.to_csv('proc_AVA.csv', index=False)

In [ ]:
len(ava_df)